In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from itertools import product
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow_model_optimization.quantization.keras import vitis_quantize

# input_horizontal =range(32,64,32)
# input_vertical =range(32,64,32)
# for input_x,input_y in product(input_horizontal,input_vertical):
shape_range =range(32,512,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    inputs = Input(shape=(shape_x, shape_y, 1))

    # 卷积层和最大池化层
    conv1 = Conv2D(64, (3,3), padding='same', activation='relu')(inputs)
    conv2 = Conv2D(64, (3,3), padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=2)(conv2)

    conv3 = Conv2D(128, (3,3), padding='same', activation='relu')(pool1)
    conv4 = Conv2D(128, (3,3), padding='same', activation='relu')(conv3)
    pool2 = MaxPooling2D(pool_size=2)(conv4)

    conv5 = Conv2D(256, (3,3), padding='same', activation='relu')(pool2)
    conv6 = Conv2D(256, (3,3), padding='same', activation='relu')(conv5)
    conv7 = Conv2D(256, (3,3), padding='same', activation='relu')(conv6)
    pool3 = MaxPooling2D(pool_size=2)(conv7)

    conv8 = Conv2D(512, (3,3), padding='same', activation='relu')(pool3)
    conv9 = Conv2D(512, (3,3), padding='same', activation='relu')(conv8)
    conv10 = Conv2D(512, (3,3), padding='same', activation='relu')(conv9)
    pool4 = MaxPooling2D(pool_size=2)(conv10)

    conv11 = Conv2D(512, (3,3), padding='same', activation='relu')(pool4)
    conv12 = Conv2D(512, (3,3), padding='same', activation='relu')(conv11)
    conv13 = Conv2D(256, (3,3), padding='same', activation='relu')(conv12)
    pool5 = MaxPooling2D(pool_size=2)(conv13)

    # 扁平层
    flat = Flatten()(pool5)

    # 全联接层
    fc1 = Dense(4096, activation='relu')(flat)
    fc2 = Dense(4096, activation='relu')(fc1)

    # 输出层
    outputs = Dense(10, activation='softmax')(fc2)


    model = Model(inputs=inputs, outputs=outputs)
    model.summary()
    model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy", 
              metrics=['accuracy'])
    #history = model.fit(x_train, y_train, epochs=1)
    #model.get_weights()[0].dtype
    quantizer = vitis_quantize.VitisQuantizer(model)
    #quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
    #quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('quantized.h5')
    !vai_c_tensorflow2 \
            --model ./quantized.h5 \
            --arch ./arch.json \
            --output_dir ./xmodel_vgg16 \
            --net_name deploy
    os.rename("./xmodel_vgg16/deploy.xmodel","./xmodel_vgg16/vgg16_{input_x}_{input_y}.xmodel".format(input_x=shape_x,input_y=shape_y))

In [ ]:
for input_x,input_y in product(input_horizontal,input_vertical):
    x_train = tf.image.resize(x_train, [input_x,input_y])
    x_test = tf.image.resize(x_test, [input_x,input_y])
    print(x_train.shape)
    print(x_train.dtype)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from itertools import product
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow_model_optimization.quantization.keras import vitis_quantize
from keras_flops import get_flops

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print('Training data: {}, {}'.format(x_train.shape, y_train.shape))
print('Test data: {}, {}'.format(x_test.shape, y_test.shape))
x_train = x_train[0:60]
y_train = y_train[0:60]
x_test = x_test[0:10]
y_test = y_test[0:10]

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

input_range =range(32,64,32)

for input_x in input_range:
    input_y = input_x
    x_train = tf.image.resize(x_train, [input_x,input_y])
    x_test = tf.image.resize(x_test, [input_x,input_y])


    inputs = Input(shape=(input_x, input_y, 1))

    # 卷积层和最大池化层
    conv1 = Conv2D(64, (3,3), padding='same', activation='relu')(inputs)
    conv2 = Conv2D(64, (3,3), padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=2)(conv2)

    conv3 = Conv2D(128, (3,3), padding='same', activation='relu')(pool1)
    conv4 = Conv2D(128, (3,3), padding='same', activation='relu')(conv3)
    pool2 = MaxPooling2D(pool_size=2)(conv4)

    conv5 = Conv2D(256, (3,3), padding='same', activation='relu')(pool2)
    conv6 = Conv2D(256, (3,3), padding='same', activation='relu')(conv5)
    conv7 = Conv2D(256, (3,3), padding='same', activation='relu')(conv6)
    pool3 = MaxPooling2D(pool_size=2)(conv7)

    conv8 = Conv2D(512, (3,3), padding='same', activation='relu')(pool3)
    conv9 = Conv2D(512, (3,3), padding='same', activation='relu')(conv8)
    conv10 = Conv2D(512, (3,3), padding='same', activation='relu')(conv9)
    pool4 = MaxPooling2D(pool_size=2)(conv10)

    conv11 = Conv2D(512, (3,3), padding='same', activation='relu')(pool4)
    conv12 = Conv2D(512, (3,3), padding='same', activation='relu')(conv11)
    conv13 = Conv2D(256, (3,3), padding='same', activation='relu')(conv12)
    pool5 = MaxPooling2D(pool_size=2)(conv13)

    # 扁平层
    flat = Flatten()(pool5)

    # 全联接层
    fc1 = Dense(4096, activation='relu')(flat)
    fc2 = Dense(4096, activation='relu')(fc1)

    # 输出层
    outputs = Dense(10, activation='softmax')(fc2)


    model = Model(inputs=inputs, outputs=outputs)
    model.summary()
    model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy", 
              metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=1)
    model.get_weights()[0].dtype
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = x_test[1:8], weight_bit=8, activation_bit=8)
    quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('tf2_mnist_classifier_quantized.h5')
    !vai_c_tensorflow2 \
            --model ./tf2_mnist_classifier_quantized.h5 \
            --arch ./arch.json \
            --output_dir ./xmodel \
            --net_name deploy
    os.rename("./xmodel/deploy.xmodel","./xmodel/mnist_classifier_vgg16_{input_x}_{input_y}.xmodel".format(input_x=input_x,input_y=input_y))

In [3]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from itertools import product
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow_model_optimization.quantization.keras import vitis_quantize

#from keras_flops import get_flops

shape_range =range(192,480,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    inputs = Input(shape=(shape_x, shape_y, 1))
    for step in range(1,11):
        # 卷积层和最大池化层
        conv1_kernel_num = random.randrange(32,64,2)
        conv1 = Conv2D(conv1_kernel_num, (3,3), padding='same', activation='relu')(inputs)
        conv2_kernel_num = random.randrange(conv1_kernel_num,64,2)
        conv2 = Conv2D(conv2_kernel_num, (3,3), padding='same', activation='relu')(conv1)
        pool1 = MaxPooling2D(pool_size=2)(conv2)

        conv3_kernel_num = random.randrange(conv2_kernel_num,128,2)
        conv3 = Conv2D(conv3_kernel_num, (3,3), padding='same', activation='relu')(pool1)
        conv4_kernel_num = random.randrange(conv3_kernel_num,128,2)
        conv4 = Conv2D(conv4_kernel_num, (3,3), padding='same', activation='relu')(conv3)
        pool2 = MaxPooling2D(pool_size=2)(conv4)

        conv5_kernel_num = random.randrange(conv4_kernel_num,256,2)
        conv5 = Conv2D(conv5_kernel_num, (3,3), padding='same', activation='relu')(pool2)
        conv6_kernel_num = random.randrange(conv5_kernel_num,256,2)
        conv6 = Conv2D(conv6_kernel_num, (3,3), padding='same', activation='relu')(conv5)
        conv7_kernel_num = random.randrange(conv6_kernel_num,256,2)
        conv7 = Conv2D(conv7_kernel_num, (3,3), padding='same', activation='relu')(conv6)
        pool3 = MaxPooling2D(pool_size=2)(conv7)

        conv8_kernel_num = random.randrange(conv7_kernel_num,512,2)
        conv8 = Conv2D(conv8_kernel_num, (3,3), padding='same', activation='relu')(pool3)
        conv9_kernel_num = random.randrange(conv8_kernel_num,512,2)
        conv9 = Conv2D(conv9_kernel_num, (3,3), padding='same', activation='relu')(conv8)
        conv10_kernel_num = random.randrange(conv9_kernel_num,512,2)
        conv10 = Conv2D(conv10_kernel_num, (3,3), padding='same', activation='relu')(conv9)
        pool4 = MaxPooling2D(pool_size=2)(conv10)

        conv11_kernel_num = random.randrange(conv10_kernel_num,512,2)
        conv11 = Conv2D(conv11_kernel_num, (3,3), padding='same', activation='relu')(pool4)
        conv12_kernel_num = random.randrange(conv11_kernel_num,512,2)
        conv12 = Conv2D(conv12_kernel_num, (3,3), padding='same', activation='relu')(conv11)
        conv13_kernel_num = random.randrange(128,256,2)
        conv13 = Conv2D(conv13_kernel_num, (3,3), padding='same', activation='relu')(conv12)
        pool5 = MaxPooling2D(pool_size=2)(conv13)

        # 扁平层
        flat = Flatten()(pool5)

        # 全联接层
        Dense1_neuron_num = random.randrange(1072,3072,32)
        fc1 = Dense(Dense1_neuron_num, activation='relu')(flat)
        Dense2_neuron_num = random.randrange(1072,Dense1_neuron_num+1,32)
        fc2 = Dense(Dense2_neuron_num, activation='relu')(fc1)

        # 输出层
        outputs = Dense(10, activation='softmax')(fc2)


        model = Model(inputs=inputs, outputs=outputs)
        model.summary()
        model.save('./vgg16_h5/vgg16_{input_x}_{input_y}_{step}.h5'.format(input_x=shape_x,input_y=shape_y,step=step))

        #model.compile(optimizer='adam',loss="sparse_categorical_crossentropy", metrics=['accuracy'])
        #history = model.fit(x_train, y_train, epochs=1)
        #model.get_weights()[0].dtype
        quantizer = vitis_quantize.VitisQuantizer(model)
        quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
        #quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
        quantized_model.save('quantized.h5')
        !vai_c_tensorflow2 \
                --model ./quantized.h5 \
                --arch ./arch.json \
                --output_dir ./xmodel_vgg16 \
                --net_name deploy
        os.rename("./xmodel_vgg16/deploy.xmodel","./xmodel_vgg16/vgg16_{input_x}_{input_y}_{step}.xmodel".format(input_x=shape_x,input_y=shape_y,step=step))

Model: "model_85"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 192, 192, 1)]     0         
_________________________________________________________________
conv2d_1118 (Conv2D)         (None, 192, 192, 50)      500       
_________________________________________________________________
conv2d_1119 (Conv2D)         (None, 192, 192, 54)      24354     
_________________________________________________________________
max_pooling2d_430 (MaxPoolin (None, 96, 96, 54)        0         
_________________________________________________________________
conv2d_1120 (Conv2D)         (None, 96, 96, 126)       61362     
_________________________________________________________________
conv2d_1121 (Conv2D)         (None, 96, 96, 126)       143010    
_________________________________________________________________
max_pooling2d_431 (MaxPoolin (None, 48, 48, 126)       0  

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 4s 474ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 8s 8s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 6s 614ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 3s 380ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 8s 8s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 67/67 [00:00<00:00, 89.21it/s]              
[INFO] dump xmodel: /tmp/deploy_org.xmodel
[UNILOG][INF

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 5s 544ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 8s 8s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

...

[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 5s 531ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 10s 10s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 67/67 [00:02<00:00, 31.74it/s]              
[INFO] dump xmodel: /tmp/deploy_org.xmodel
[UNILOG][I

[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 4s 465ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 67/67 [00:01<00:00, 38.62it/s]              
[INFO] dump xmodel: /tmp/deploy_org.xmodel
[UNILOG][INF

[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 4s 497ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 67/67 [00:02<00:00, 28.80it/s]              
[INFO] dump xmodel: /tmp/deploy_org.xmodel
[UNILOG][INF

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 7s 804ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 12s 12s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel    

KeyboardInterrupt: 